In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../LSTM Karimnagar AQI.csv")
df

,Unnamed: 0,Date,PM10,NOx,SO2
0,0,31-01-2023,87.539910,27.158888,7.933721
1,1,28-02-2023,81.372370,25.184647,7.803687
2,2,31-03-2023,80.789470,29.783690,7.582243
3,3,30-04-2023,67.225900,29.469425,7.051498
4,4,31-05-2023,50.658237,24.970583,6.584076
5,5,30-06-2023,58.736134,24.321030,6.474168
6,6,31-07-2023,65.171100,22.901304,7.026757
7,7,31-08-2023,72.050350,28.899277,7.698150
8,8,30-09-2023,88.017815,29.130520,7.624876
9,9,31-10-2023,96.375610,32.064262,7.587214


In [3]:
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

df["PM10_SubIndex"] = df["PM10"].apply(lambda x: get_PM10_subindex(x))

In [4]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

df["SO2_SubIndex"] = df["SO2"].apply(lambda x: get_SO2_subindex(x))

In [5]:
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

df["NOx_SubIndex"] = df["NOx"].apply(lambda x: get_NOx_subindex(x))

In [6]:
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

df["AQI_calculated"] = round(df[["PM10_SubIndex", "SO2_SubIndex", "NOx_SubIndex"]].max(axis = 1))

df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df[~df.AQI_calculated.isna()].head(12)


,Unnamed: 0,Date,PM10,NOx,SO2,PM10_SubIndex,SO2_SubIndex,NOx_SubIndex,AQI_calculated,AQI_bucket_calculated
0,0,31-01-2023,87.539910,27.158888,7.933721,87.539910,9.917151,33.948610,88.0,Satisfactory
1,1,28-02-2023,81.372370,25.184647,7.803687,81.372370,9.754608,31.480809,81.0,Satisfactory
2,2,31-03-2023,80.789470,29.783690,7.582243,80.789470,9.477803,37.229613,81.0,Satisfactory
3,3,30-04-2023,67.225900,29.469425,7.051498,67.225900,8.814372,36.836781,67.0,Satisfactory
4,4,31-05-2023,50.658237,24.970583,6.584076,50.658237,8.230096,31.213229,51.0,Satisfactory
5,5,30-06-2023,58.736134,24.321030,6.474168,58.736134,8.092710,30.401288,59.0,Satisfactory
6,6,31-07-2023,65.171100,22.901304,7.026757,65.171100,8.783446,28.626630,65.0,Satisfactory
7,7,31-08-2023,72.050350,28.899277,7.698150,72.050350,9.622687,36.124096,72.0,Satisfactory
8,8,30-09-2023,88.017815,29.130520,7.624876,88.017815,9.531095,36.413150,88.0,Satisfactory
9,9,31-10-2023,96.375610,32.064262,7.587214,96.375610,9.484018,40.080327,96.0,Satisfactory


In [7]:

df = df[['Date', 'AQI_calculated', 'AQI_bucket_calculated']]

In [8]:
df

,Date,AQI_calculated,AQI_bucket_calculated
0,31-01-2023,88.0,Satisfactory
1,28-02-2023,81.0,Satisfactory
2,31-03-2023,81.0,Satisfactory
3,30-04-2023,67.0,Satisfactory
4,31-05-2023,51.0,Satisfactory
5,30-06-2023,59.0,Satisfactory
6,31-07-2023,65.0,Satisfactory
7,31-08-2023,72.0,Satisfactory
8,30-09-2023,88.0,Satisfactory
9,31-10-2023,96.0,Satisfactory


In [9]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Karimnagar AQI.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)

Mounted at /content/drive
